# Python emojipedia scraper
> By [@aadityasiva](https://kaggle.com/aadityasiva)

## Installing the required packages

In [1]:
!pip install requests beautifulsoup4 pandas

## Importing the needed packages

In [2]:
import requests
import bs4
import pandas as pd

## Loading the website soup

In [3]:
baseurl = "https://emojipedia.org"
res = requests.get(baseurl)
soup = bs4.BeautifulSoup(res.text, "html5lib")

## Finding all the divs on the page with block class

In [4]:
allDivs = soup.find_all("div", {"class": "block"})

## Getting the urls from the first div with class block

In [5]:
categoriesDiv = allDivs[0]
categoriesLinks = []
for listItem in categoriesDiv.find_all('li'):
  for a in listItem.find_all('a', href=True):
    categoriesLinks.append(a['href'])
    print("Found the URL:", a['href'])

Found the URL: /people/
Found the URL: /nature/
Found the URL: /food-drink/
Found the URL: /activity/
Found the URL: /travel-places/
Found the URL: /objects/
Found the URL: /symbols/
Found the URL: /flags/


## Finding all the emoji links

In [6]:
emojiLinks = []
for categoryLink in categoriesLinks:
  linkToVisit = (baseurl + categoryLink)
  categoryRes = requests.get(linkToVisit)
  categorySoup = bs4.BeautifulSoup(categoryRes.text, "html5lib")
  for emojiLinksList in categorySoup.find_all('ul', {"class": "emoji-list"}):
    for links in emojiLinksList.find_all('a', href=True):
      emojiLinks.append(links['href'])

## Extracting the emoji, emoji name, emoji description

In [7]:
data = {
    "emojis": [],
    "emojiNames": [],
    "emojiDescriptions": []
}
for emojiLink in emojiLinks:
  linkToVisit = (baseurl + emojiLink)
  emojiRes = requests.get(linkToVisit)
  emojiSoup = bs4.BeautifulSoup(emojiRes.text, 'html5lib')
  for article in emojiSoup.find_all('article'):
    emojiName = (article.find('h1').text[2:])
    emoji = (article.find('span', 'emoji').text)
  for description in emojiSoup.find_all('section', {"class": "description"}):
    data['emojiDescriptions'].append(description.find('p').getText())
  data['emojis'].append(emoji)
  data['emojiNames'].append(emojiName)

## Exporting the scraped data as csv and json

In [8]:
df = pd.DataFrame(data)
df.to_csv('data.csv')
df.to_json('data.json')